In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
county_sub_submkt_mapping = pd.read_csv('/mnt/container1/np_forecast_data/submarket_county_sub_mapping.csv')
county_sub_submkt_mapping.head(50)

,research_market,research_submkt_id,pct_area,geoid,statefp,countyfp,cousubfp,county_geoid
0,Phoenix,LIN114,1.00,400791277,4,7,91277,4007
1,Phoenix,LIN114,1.00,400791495,4,7,91495,4007
2,Phoenix,LIN114,1.00,400792570,4,7,92570,4007
3,Phoenix,LIN114,1.00,400793109,4,7,93109,4007
4,Phoenix,LIN114,1.00,400793470,4,7,93470,4007
5,Phoenix,LIN114,1.00,400793927,4,7,93927,4007
6,Phoenix,PHO052,0.80,401390459,4,13,90459,4013
7,Phoenix,PHO052,0.53,401390867,4,13,90867,4013
8,Phoenix,PHO052,0.90,401391377,4,13,91377,4013
9,Phoenix,PHO052,0.06,401392601,4,13,92601,4013


In [8]:
pho_countysub_fips = county_sub_submkt_mapping[(county_sub_submkt_mapping['research_submkt_id']!='LIN114')&(county_sub_submkt_mapping['research_market']=='Phoenix')]['cousubfp'].unique().tolist()
dal_countysub_fips = county_sub_submkt_mapping[(county_sub_submkt_mapping['research_submkt_id']!='LIN076')&(county_sub_submkt_mapping['research_market']=='Dallas-Fort Worth')]['cousubfp'].unique().tolist()

pho_countysub_fips = [str(fip) for fip in pho_countysub_fips]
dal_countysub_fips = [str(fip) for fip in dal_countysub_fips]

pho_countysub_fips_str = ','.join(pho_countysub_fips)
dal_countysub_fips_str = ','.join(dal_countysub_fips)
pho_countysub_fips

['90459',
 '90867',
 '91377',
 '92601',
 '93060',
 '93472',
 '93774',
 '90510',
 '90816',
 '91173',
 '91224',
 '91428',
 '92091',
 '92346',
 '93162',
 '93377',
 '90561',
 '93009']

In [83]:


def get_countysub_data(
    year: str=None,
    variables: str=None,
    state_fips: str=None,
    countysub_fips: list=None):

    base_url = 'https://api.census.gov/data'
    dataset = 'acs/acs1'

    if year is None:
        print('Please input variable: year.')
    year = year
    
    if variables is None:
        print('Please input variable: variables.')
    variables = variables
    
    level = 'county%20subdivision'
    
    if state_fips is None:
        print('Please input varaible: state_fips.')

    state_fips = state_fips
    
        
    if countysub_fips is None:
        print('Please input varaible: countysub_fips.')
        
    countysub_fips = countysub_fips  

    api_key = '2caac992889c23e9fe800fc4a11ffd2248146d2c'  

    url = f'{base_url}/{year}/{dataset}/profile?get={variables}&for={level}:*&in=state:{state_fips}&in=county:*&key={api_key}'


    r = requests.get(url)

    try:
        resp = r.json()
        if r.status_code == 200:
            data = resp[1:]
            df = pd.DataFrame(data, columns=resp[0])
            df['countysub_geoid'] = df['state'] + df['county'] + df['county subdivision']
            df['countysub_geoid'] = df['countysub_geoid'].astype(str).apply(lambda x: int(x[1:]) if x.startswith('0') else int(x))
            dt = f'{year}-01-01'
            df['date'] = pd.Series([dt] * df.shape[0])
            df['date'] = pd.to_datetime(df['date'])
            df = df[df['county subdivision'].isin(countysub_fips)]
            df = df.drop(['state', 'county', 'county subdivision'], axis=1)

            return df
        else:
            print("Error occurred while retrieving data.")
    except ValueError: 
        print(f"{year}:Response content is not valid JSON.")
        
        


In [ ]:
def get_asc_countysub_data(year, variables, state_fips, countsub_fips):
    dfs = pd.DataFrame()

    for yr in year:
        df = get_countysub_data(yr, variables, state_fips, countsub_fips)
        dfs = pd.concat([dfs, df])

    return dfs

In [7]:

#pho_fips_codes_str = '013,021,019'
#dal_fips_codes_str = '231,213,119,121,139,257,085,113,397,439,497,251,367,221'
year = [str(yr) for yr in range(2005, 2022)]
variables = 'B01001_001E'
state_fips = '04'
#county_fips = pho_fips_codes_str
countysub_fips = pho_countysub_fips
#pho_cs_df = get_asc_countysub_data(year, variables, state_fips, countysub_fips)
#pho_cs_df.head()
#pho_df.to_csv("/mnt/container1/zqiao_Workspace/link-research/ad-hoc/zq-sandbox/submkt_data/pho_csa_county_sub.csv")



2005:Response content is not valid JSON.
2006:Response content is not valid JSON.
2007:Response content is not valid JSON.
2008:Response content is not valid JSON.
2009:Response content is not valid JSON.
2010:Response content is not valid JSON.
2011:Response content is not valid JSON.
2012:Response content is not valid JSON.
2013:Response content is not valid JSON.
2014:Response content is not valid JSON.
2015:Response content is not valid JSON.
2016:Response content is not valid JSON.
2017:Response content is not valid JSON.
2018:Response content is not valid JSON.
2019:Response content is not valid JSON.
2020:Response content is not valid JSON.
2021:Response content is not valid JSON.


""


In [ ]:
# get Phoenix county_subdivision population data
state_fips = '48'
county_fips = dal_fips_codes_str
countsub_fips = pho_countsub_fips

dal_cs_df = get_asc_blockgroup_data(year, variables, state_fips, county_fips, countsub_fips)
dal_cs_df.head()
#dal_df.to_csv("/mnt/container1/zqiao_Workspace/link-research/ad-hoc/zq-sandbox/submkt_data/dal_csa_county_sub.csv")
